In [1]:
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import json
import pickle
import csv
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import networkx as nx
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re
import pandas
import pandas as pd
import tweepy
import base64
import datetime
import preprocessor as p
from tweepy import Stream
from sklearn.externals import joblib
from nltk import ngrams
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from node2vec import Node2Vec

Using TensorFlow backend.
/home/sakshi18013/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sakshi18013/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sakshi18013/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sakshi18013/anaconda3/lib/python3.7

In [3]:
train_data=dict()

In [5]:
def getRandomtweetAPI():
    import random
    df = pandas.read_csv(os.getcwd()+'/tokens_random.csv',encoding='ISO-8859-1',keep_default_na=False)    
    totalTokens=df.shape[0]
    index=random.randint(0,totalTokens-1)   
    ckey=df['ConsumerKey'][index]
    csecret=df['ConsumerSecret'][index]
    atoken=df['TwitterToken'][index]
    asecret=df['TwitterSecret'][index]
    app_only_auth: true
    return ckey,csecret,atoken,asecret

In [6]:
def get_api():
    ckey,csecret,atoken,asecret=getRandomtweetAPI()
    auth = OAuthHandler(ckey, csecret)
    auth.set_access_token(atoken, asecret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

# Prepare Dataset

In [4]:

with open('follow_network_with_id.pkl', 'rb') as handle:
    foll_id = pickle.load(handle)

In [13]:
with open('Retweeter_dict_time.pickle', 'rb') as handle:
    rts_time = pickle.load(handle)

In [3]:

with open('Retweeter_dict.pickle', 'rb') as handle:
    rts = pickle.load(handle)

In [10]:
# with open('tweet_features.pickle', 'rb') as handle:
#     tweet_features = pickle.load(handle)
# retweet_features=dict()
# retweeters_dataset_dict=dict()

In [9]:
G= nx.DiGraph()
import copy

In [12]:
# FOLDERNAME1="twitter_Retweeters_history"
# FOLDERNAME2="twitter_Followers_history"
# rts_dataset=dict()
# total=0
# labels=[]
# ct=0
# for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
#     for fname in files:
# #         print(fname)
#         if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
#             filepath=root+"/"+fname
#             search_file=filepath.split("/")
# #             print("here")
#             if len(search_file)>7:
# #                 print(search_file[7])
#                 tweets=dict()
#                 with open(filepath) as csv_file:
#                     csv_reader= csv.DictReader(csv_file)
#                     for row in csv_reader:

#                         inner={}
#                         if  int(row['retweet_count'])>5:
# #                             row['id_'] in tweet_features and
                            
#                             if int(row['user_id']) in foll_id  and int(row['id_']) in rts:
#                                 retweeters = rts[int(row['id_'])]
                
#                                 tobecovered=copy.deepcopy(retweeters)
# # first_retweets=[]
#                                 print("level1 passed",row['id_'],int(row['retweet_count']))                       

# ####FOR FIRS LEVEL FOLLOWERS
# # for value in rts[int(row['id_'])]:
#                                 post_user = row['user_id']
#                                 for value in foll_id[int(row['user_id'])]:
# #                                     print(post_user, value)
                    
#                                     G.add_edge(post_user , value)
                                  
# ####FOR SECOND LEVEL FOLLOWERS  
# #                                 print(type(foll_id[int(row['user_id'])]), type(retweeters))
#                                 level1 = list(set(foll_id[int(row['user_id'])]) and  set(retweeters))
# #                                 print((level1) , "First level")
#                                 l2=list(set(retweeters)-set(level1))
# #                                 print(l2)
# #                                 second_retweets=[]
# #                                 print(len(level)
                                
#                                 l2_covered=[]
#                                 if l2:
#                                     for user in level1:
#     # #                                     print(" user",user)
#                                         if int(user) in foll_id:

#                                             for value1 in foll_id[int(user)]:
#     #                                                 print("here")
#                                                     if value1 in l2:
#                                                         l2_covered.append(value1)
#                                                     G.add_edge(user ,value1)
#                                                     ct+=1
                
                                
#         # ####THIRD LEVEL FOLLOWERS
#                                     level3 = list(set(l2)-set(l2_covered))
#     #                                 third_retweets=[]
#     # #                                 print("IN LEVEL 3",level3)
#                                     print(len(level3) , "Secomd level")
#                                     l3 =list( set(retweeters )- set(level3))

#                                     l3_covered=[]
#                                     if l3:
#                                         for user3 in level3:
#                                             if int(user3) in foll_id:

#                                                 for value2 in foll_id[int(user3)]:
#         #                                                 print("here")
#                                                         if value2 in l3:
#                                                             l3_covered.append(value2)
#                                                         G.add_edge(user3 ,value2)
#                                                         ct+=1

                                
#                                         level4= list(set(l3)-set(l3_covered))
#                                         G.add_node("Dummy")
#                                         G.add_edge(post_user , "Dummy")
#                                         for v in level4:
#                                             G.add_edge("Dummy", v)
#                                             ct+=1
#                                 print("Total number of retweeters:" , len(retweeters) ,"\n remainng nodes dumy connected" , len(retweeters)-ct)
#         #                                     
    
# #                     break 

In [6]:
G= nx.DiGraph()
import copy

In [14]:
# G= nx.DiGraph()
# FOLDERNAME1="twitter_Retweeters_history"
# FOLDERNAME2="twitter_Followers_history"
# rts_dataset=dict()
# total=0
# labels=[]
# all_paths = []
# for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
#     for fname in files:
# #         print(fname)
#         if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
#             filepath=root+"/"+fname
#             search_file=filepath.split("/")
# #             print("here")
#             if len(search_file)>7:
# #                 print(search_file[7])
#                 tweets=dict()
#                 with open(filepath) as csv_file:
#                     csv_reader = csv.DictReader(csv_file)
#                     for row in csv_reader:
                        
#                         inner={}
#                         if  int(row['retweet_count'])>5:
# #                             row['id_'] in tweet_features and
#                             G1= nx.DiGraph()
#                             if int(row['user_id']) in foll_id  and int(row['id_']) in rts:
#                                 retweeters = rts[int(row['id_'])]
#                                 tobecovered=[]
#                                 tobecovered=copy.deepcopy(retweeters)
# # first_retweets=[]
# #                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       

# ####FOR FIRS LEVEL FOLLOWERS
# # for value in rts[int(row['id_'])]:
#                                 post_user = row['user_id']
#                                 for value in foll_id[int(row['user_id'])]:
# #                                     print(post_user, value)
#                                     try:
        
#                                         G1.add_edge(post_user , value)
#                                     except:
#                                         pass
                                  
# ####FOR SECOND LEVEL FOLLOWERS  
# #                                 print(type(foll_id[int(row['user_id'])]), type(retweeters))
#                                 level1 = list(set(foll_id[int(row['user_id'])]) and  set(retweeters))
# #                                 print("covered in level1",len(level1))
# #                                 print((level1) , "First level")
#                                 l2=list(set(retweeters)-set(level1))
#                                 for i in level1: 
#                                     try: 
#                                         tobecovered.remove(i) 
#                                     except ValueError: 
#                                         pass
# #                                 tobecovered.remove(level1)
# #                                 if(len(tobecovered)>0):
# #                                     print(len(tobecovered),"rem 2")
# #                                 print(len(l2), len(tobecovered) , "check")
# #                                 second_retweets=[]
# #                                 print(len(level)
                                
#                                 l2_covered=[]
#                                 if tobecovered:
# #                                     print("in")
#                                     for user in level1:
#     # #                                     print(" user",user)
#                                         if int(user) in foll_id:
# #                                             print("got")

#                                             for value1 in foll_id[int(user)]:
#     #                                                 print("here")
#                                                     if value1 in tobecovered:
#                                                         l2_covered.append(value1)
# #                                                     print("adding")
#                                                     G1.add_edge(user ,value1)
# #                                                     if nx.is_
                
#                                     for i in l2_covered: 
#                                         try: 
#                                             tobecovered.remove(i) 
#                                         except ValueError: 
#                                             pass
# #                                     if(len(tobecovered)>0):
# #                                         print(len(tobecovered),"rem 3")
# #                                     tobecovered.remove(l2_covered)
#         # ####THIRD LEVEL FOLLOWERS
#                                     level3 = list(set(l2)-set(l2_covered))
#     #                                 third_retweets=[]
#     # #                                 print("IN LEVEL 3",level3)
# #                                     print(len(level3) , "Secomd level")
#                                     l3 =list( set(retweeters )- set(level3))
#                                     for i in level3: 
#                                         try: 
#                                             tobecovered.remove(i) 
#                                         except ValueError: 
#                                             pass
# #                                     tobecovered.remove(level3)
#                                     l3_covered=[]
#                                     if tobecovered:
#                                         for user3 in level3:
#                                             if int(user3) in foll_id:

#                                                 for value2 in foll_id[int(user3)]:
#         #                                                 print("here")
#                                                         if value2 in tobecovered:
#                                                             l3_covered.append(value2)
#                                                         G1.add_edge(user3 ,value2)
#                                         for i in l3_covered: 
#                                             try: 
# #                                                 print("try")
#                                                 tobecovered.remove(i) 
#                                             except ValueError: 
#                                                 pass
                                    
# #                                         tobecovered.remove(l3_covered)
# #                                         if(len(tobecovered)>0):
# #                                             print(len(tobecovered),"rem 4")
                                
#                                         level4= list(set(l3)-set(l3_covered))
#                                         if level4:
#                                             G1.add_node("Dummy")
#                                             G1.add_edge(post_user , "Dummy")
#                                             for v in level4:
#                                                 G1.add_edge("Dummy", v)
#                                             for i in level4: 
#                                                 try: 
#                                                     tobecovered.remove(i) 
#                                                 except ValueError: 
#                                                     pass
# #                                             tobecovered.remove(level4)
#                                             print("Dummy Nodes:" , len(level4))
#                                 print("Total number of retweeters:" , len(retweeters) )
# #         ,"\n remainng nodes dumy connected" , len(level4))
# #                                 print("Remaining: " , len(tobecovered))              
    
# #                     break 
#                             print(len(G1.nodes), len(G1.edges()) , nx.is_directed_acyclic_graph(G1))
#                             for v , d in G1.in_degree() :
#                                 if d==0:
#                                     print(v)
#                             roots = (v for v, d in G1.in_degree() if d == 0)
#                             leaves = (v for v, d in G1.out_degree() if d == 0)

#                             for root in roots:
#                                 print("Root:",root)
#                                 for leaf in leaves:
# #                                     print("Leaf",leaf)
#                                     paths = nx.all_simple_paths(G1, root, leaf)
#                                     all_paths.extend(paths)
#     #                                 break
#                             print(len(all_paths))
# #                             break
                        

In [15]:
print(len(G.nodes()) ,len(G.edges()))

0 0


In [16]:
# print(all_paths[26845:29582])

# GRAPH CREATION

In [76]:
# G= nx.DiGraph()
# FOLDERNAME1="twitter_Retweeters_history"
# FOLDERNAME2="twitter_Followers_history"
# rts_dataset=dict()
# total=0
# labels=[]
# all_paths = []
# all_path2=[]
# final_cascade =[]
# final_test_cascade=[]
# max_=0
# count_=0
# for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#IndiaBoycottsNPR'):
#     for fname in files:
# #         print(fname)
#         if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
#             filepath=root+"/"+fname
#             search_file=filepath.split("/")
# #             print("here")
#             if len(search_file)>7:
# #                 print(search_file[7])
#                 tweets=dict()
#                 with open(filepath) as csv_file:
#                     csv_reader = csv.DictReader(csv_file)
#                     for row in csv_reader:
                        
#                         inner={}
#                         if  int(row['retweet_count'])>35:
# #                             row['id_'] in tweet_features and
# #                             G1= nx.DiGraph()
# #                             G2 = nx.DiGraph()

#                             if int(row['user_id']) in foll_id  and int(row['id_']) in rts:
#                                 retweeters = rts[int(row['id_'])]
# #                                 tobecovered=set()
#                                 tobecovered=set(copy.deepcopy(retweeters))
# # first_retweets=[]
# #                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       
            
# ####FOR FIRS LEVEL FOLLOWERS
# # for value in rts[int(row['id_'])]:
#                                 post_user =row['user_id']
#                                 print(post_user)
#                                 for value in foll_id[int(row['user_id'])]:
# #                                     print(post_user, value)
#                                     try:
        
#                                         G.add_edge(post_user , value)
#                                     except:
#                                         pass
                    
#                                 upper_level = set([post_user])
#                                 flag = False
            
#                                 iter_=1
# #                                 print("before loop: {}".format(foll_id[int(post_user)]))
                                
#                                 while(not flag and tobecovered):
#                                     print("iter: {}".format(iter_))
#                                     iter_+=1
#                                     flag=True
#                                     nowcovered=set()
# #                                     followers=set()
#                                     for each_user in upper_level: 
#                                         if int(each_user) in foll_id:
#                                             for each_follower in foll_id[int(each_user)]:
# #                                                 followers.add(each_follower)
# #                                                 G.add_edge(each_user , each_follower)
#                                                 if each_follower in tobecovered:
#                                                     nowcovered.add(each_follower)
#                                                     flag=False
#                                                     G.add_edge(each_user , each_follower)
#                                     for each_covered in nowcovered:
#                                         tobecovered.remove(each_covered)
#                                     upper_level = nowcovered
# #                                 print(flag , len(tobecovered),"after while")
                                
#                                 if tobecovered:
#                                     for v in tobecovered:
#                                         G.add_node(v)
#                                     upper_level = copy.deepcopy(tobecovered)
#                                     flag = False
#                                     iterg_=1
#     #                                 print("before loop: {}".format(foll_id[int(post_user)]))

#                                     while(not flag and tobecovered):
#                                         print("iter: {}".format(iterg_))
#                                         iterg_+=1
#                                         flag=True
#                                         nowcovered=set()
# #                                         followers=set()
#                                         for each_user in upper_level:
#                                             if int(each_user) in foll_id:
#                                                 for each_follower in foll_id[int(each_user)]:
# #                                                     followers.add(each_follower)
# #                                                     G.add_edge(each_user , each_follower)
#                                                     if each_follower in tobecovered:
#                                                         nowcovered.add(each_follower)
#                                                         flag=False
#                                                         G.add_edge(each_user , each_follower)
#                                         for each_covered in nowcovered:
#                                             tobecovered.remove(each_covered)
#                                         upper_level = nowcovered
# #                                     print(flag , len(tobecovered),"after 2nd while")
                                    
                                    


# #                             roots = (v for v, d in G1.in_degree() if d == 0)
# #                             leaves = (v for v, d in G1.out_degree() if d == 0)

# #                             for root in roots:
# # #                                 print("Root:",root)
# #                                 for leaf in leaves:
# # #                                     print("Leaf",leaf)
# #                                     paths = nx.all_simple_paths(G1, root, leaf)
# #                                     all_paths.extend(paths)
# #     #                                 break
# #                             print("Actual cacades:" , len(all_paths))
        
        
# #                             roots = (v for v, d in G2.in_degree() if d == 0)
# #                             leaves = (v for v, d in G2.out_degree() if d == 0)

# #                             for root in roots:
# # #                                 print("Root:",root)
# #                                 for leaf in leaves:
# # #                                     print("Leaf",leaf)
# #                                     paths = nx.all_simple_paths(G2, root, leaf)
# #                                     all_path2.extend(paths)
# #     #                                 break
# #                             max_+=len(all_path2)
# #                             count_+=1
# #                             print(max_)
# #     #                             break
#     #                             dummy_path = all_path2 + list(tobecovered)
#     #                             print("Dummy cascade" , len(dummy_path))
# #                             print(dummy_path)


# #                             for actual in all_paths:
# #                                 final_test_cascade.append(actual)
# #                                 if len(all_path2)==0  or len(all_path2)<71:
# #                                     final_cascade.append(actual)
# #                                 else:
# #                                     for dummy in all_path2:
# #                                         temp=list()
# #                                         temp.extend(actual)
# #                                         temp.extend(dummy)
# #                                         final_cascade.append(temp)

# #                             break
                        

81364970
iter: 1
iter: 2
iter: 3
iter: 1
iter: 2
1190987300492079104
iter: 1
iter: 2
iter: 1
702485299
iter: 1
iter: 2
iter: 1
4528824494
iter: 1
iter: 2
iter: 3
iter: 1
37370011
iter: 1
iter: 2
iter: 3
iter: 1
iter: 2
1190987300492079104
iter: 1
iter: 2
iter: 3
iter: 4
iter: 1
iter: 2
1127527776188190722
iter: 1
iter: 2
iter: 3
iter: 4
iter: 1
1127527776188190722
iter: 1
iter: 2
iter: 1
iter: 2
1190987300492079104
iter: 1
iter: 2
iter: 3
iter: 1
iter: 2
1127527776188190722
iter: 1
iter: 2
iter: 3
iter: 1
1236514372031098881
iter: 1
iter: 1
iter: 2
1127527776188190722
iter: 1
iter: 2
iter: 3
iter: 4
iter: 1
iter: 2
1127527776188190722
iter: 1
iter: 2
iter: 3
iter: 4
iter: 1
iter: 2
1127527776188190722
iter: 1
iter: 2
iter: 3
iter: 4
iter: 1


In [79]:
print(len(G.nodes) , len(G.edges()))

18473 21538


In [80]:
# nx.write_gpickle(G , "final_complete_graph_npr2")

# CASCADE CREATION

In [57]:
# G= nx.DiGraph()
FOLDERNAME1="twitter_Retweeters_history"
FOLDERNAME2="twitter_Followers_history"
rts_dataset=dict()
total=0
final_data=[]
labels=[]
all_paths = []
all_path2=[]
final_cascade =[]
final_test_cascade=[]
max_=0
count_=0
for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
    for fname in files:
#         print(fname)
        if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
            filepath=root+"/"+fname
            search_file=filepath.split("/")
#             print("here")
            if len(search_file)>7:
#                 print(search_file[7])
                tweets=dict()
                with open(filepath) as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
                        
                        inner={}
                        if  int(row['retweet_count'])>10:
#                             row['id_'] in tweet_features and
                            G1= nx.DiGraph()
                            G2 = nx.DiGraph()

                            if int(row['user_id']) in foll_id  and int(row['id_']) in rts and int(row['id_']) in rts_time:
                                retweeters = rts[int(row['id_'])]
#                                 tobecovered=set()
                                tobecovered=set(copy.deepcopy(retweeters))
                                print(len(tobecovered) , "tobecovered")
                                print(tobecovered)
# first_retweets=[]
#                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       
            
####FOR FIRS LEVEL FOLLOWERS
# for value in rts[int(row['id_'])]:
                                post_user =row['user_id']
                                upper_level = set([post_user])
                                flag = False
                                iter_=1
#                                 print("before loop: {}".format(foll_id[int(post_user)]))
                                
                                while(not flag and tobecovered):
                                    print("iter: {}".format(iter_))
                                    iter_+=1
                                    flag=True
                                    nowcovered=set()
                                    for each_user in upper_level:
                                        if int(each_user) in foll_id:
                                            for each_follower in foll_id[int(each_user)]:
                                                if each_follower in tobecovered:
                                                    nowcovered.add(each_follower)
                                                    flag=False
                                                    G1.add_edge(each_user , each_follower)
                                    for each_covered in nowcovered:
                                        tobecovered.remove(each_covered)
                                    upper_level = nowcovered
                                print(flag , len(tobecovered),"after while")
                                
                                if tobecovered:
                                    upper_level = copy.deepcopy(tobecovered)
                                    flag = False
                                    iterg_=1
    #                                 print("before loop: {}".format(foll_id[int(post_user)]))

                                    while(not flag and tobecovered):
                                        print("iter: {}".format(iterg_))
                                        iterg_+=1
                                        flag=True
                                        nowcovered=set()
                                        for each_user in upper_level:
                                            if int(each_user) in foll_id:
                                                for each_follower in foll_id[int(each_user)]:
                                                    if each_follower in tobecovered:
                                                        nowcovered.add(each_follower)
                                                        flag=False
                                                        G2.add_edge(each_user , each_follower)
                                        for each_covered in nowcovered:
                                            tobecovered.remove(each_covered)
                                        upper_level = nowcovered
                                    print(flag , len(tobecovered),"after 2nd while")
                                    
                                    


                                roots = (v for v, d in G1.in_degree() if d == 0)
                                leaves = (v for v, d in G1.out_degree() if d == 0)

                                for root in roots:
                                    for leaf in leaves:
                                        paths = nx.all_simple_paths(G1, root, leaf)
                                        all_paths.extend(paths)
                                
                                roots = (v for v, d in G2.in_degree() if d == 0)
                                leaves = (v for v, d in G2.out_degree() if d == 0)

                                for root in roots:
    #                                 print("Root:",root)
                                    for leaf in leaves:
    #                                     print("Leaf",leaf)
                                        paths = nx.all_simple_paths(G2, root, leaf)
                                        all_path2.extend(paths)
        #                                 break
                                max_+=len(all_path2)
                                count_+=1
                                print(max_)
        #                             break
        #                             dummy_path = all_path2 + list(tobecovered)
        #                             print("Dummy cascade" , len(dummy_path))
    #                             print(dummy_path)


                                for actual in all_paths:
                                    final_test_cascade.append(actual)
                                    if len(all_path2)==0  or len(all_path2)<71:
                                        final_cascade.append(actual)
                                    else:
                                        for dummy in all_path2:
                                            temp=list()
                                            temp.extend(actual)
                                            temp.extend(dummy)
                                            final_cascade.append(temp)
#                                 print("Actual cacades:" , len(final_cascade))
                                arr = rts_time[int(row['id_'])]
                                for path in final_cascade:
                                    if len(path)>5:
                                        print(path)
                                        temp={}
                                        for element in path[1:]:
                                            print(arr.keys())
                                            print(len(arr.keys()) ," ret time")
                                            temp[element]=arr[element]
                                        sorted_temp ={k: v for k, v in sorted(temp.items(), key=lambda item: item[1])}
                                        inner_data=[path[0]]
                                        [inner_data.extend([k,v]) for k,v in sorted_temp.items()]
                                        final_data.append(inner_data)
                                    
                            


32 tobecovered
{'837516459987386369', '2178214820', '864952746285252608', '795681622364450816', '1171452867313463298', '947173998709817345', '926698045701464065', '1215642777611976709', '923892620178235392', '1677723938', '1118713226160553985', '74810575', '1146249629509439488', '2698033460', '16804541', '3232492459', '1165358470129954816', '2307712298', '2307132318', '948737821262544901', '1857092954', '731439265092128768', '75605073', '1190683107160051712', '724590194138046465', '1185580744095092737', '44616260', '1071999249368309760', '361913249', '888069606161735681', '1164003523442302976', '614908818'}
iter: 1
iter: 2
True 7 after while
iter: 1
True 7 after 2nd while
0
11 tobecovered
{'773882650041909248', '2870252089', '142393780', '52138610', '1229295193750298624', '416397821', '1370038970', '873369085496741889', '1007854050', '774841434478219264', '1207392505928527872'}
iter: 1
iter: 2
True 1 after while
iter: 1
True 1 after 2nd while
0
34 tobecovered
{'2158989308', '974190180'

True 21 after while
iter: 1
True 21 after 2nd while
587
17 tobecovered
{'2158989308', '1141932005824688129', '1212529838776451072', '1476145002', '1077896735995908096', '894651337891168256', '2738005944', '1047039933456769024', '1075713984370749440', '974684543159451648', '935142655213703168', '1208661269810561024', '76015358', '999673255605633026', '1010023387278659584', '447821112', '1090950762681774089'}
iter: 1
True 17 after while
iter: 1
iter: 2
True 1 after 2nd while
657
11 tobecovered
{'964312837', '1736633192', '1125116300894855171', '1182544631822602240', '790972165415841792', '1038743499032514560', '895472378335051776', '231447773', '260156141', '537970070', '75277295'}
iter: 1
iter: 2
True 1 after while
iter: 1
True 1 after 2nd while
727
19 tobecovered
{'1149665089839980553', '4385757680', '1218608603516502017', '1219792845474680832', '843715399199997953', '1188873335481876481', '1620411824', '1003681975771783168', '836963486135631872', '867641215335882752', '783322999894052

KeyError: '1188504743103979520'

In [44]:
1229834046768730112 in rts_time

False

In [47]:
final_data

[]

In [45]:
final_test_cascade

[['2826529099', '2270213274'],
 ['2826529099', '42779085'],
 ['2826529099', '1138786355218145281'],
 ['2826529099', '2328865356'],
 ['2826529099', '3903318556'],
 ['2826529099', '448764973'],
 ['2826529099', '785904386799120384'],
 ['2826529099', '787467474'],
 ['2826529099', '2270213274'],
 ['2826529099', '42779085'],
 ['2826529099', '1138786355218145281'],
 ['2826529099', '2328865356'],
 ['2826529099', '3903318556'],
 ['2826529099', '448764973'],
 ['2826529099', '785904386799120384'],
 ['2826529099', '787467474'],
 ['2826529099', '2270213274'],
 ['2826529099', '42779085'],
 ['2826529099', '1138786355218145281'],
 ['2826529099', '2328865356'],
 ['2826529099', '3903318556'],
 ['2826529099', '448764973'],
 ['2826529099', '785904386799120384'],
 ['2826529099', '787467474'],
 ['1076486409542873088', '1215642777611976709'],
 ['1076486409542873088', '947173998709817345'],
 ['1076486409542873088', '724590194138046465'],
 ['1076486409542873088', '1190683107160051712'],
 ['1076486409542873088'

In [75]:
import pickle
with open('train_cascade_npr2.pkl', 'wb') as f:
    pickle.dump(final_cascade, f)
    
with open('test_cascade_npr2.pkl', 'wb') as f:
    pickle.dump(final_test_cascade, f)

In [42]:
# for each in final_cascade[1587045]:
#     print(each)
# # print(len(final_cascade[5000]))
# # print(len(set(final_cascade[5000])))

1213812904413122563
1146802871431655428
935142655213703168
400184218


In [66]:
len(final_test_cascade)

8

In [43]:
nx.is_directed_acyclic_graph(G)

True

In [ ]:
# roots = (v for v, d in G.in_degree() if d == 0)
# leaves = (v for v, d in G.out_degree() if d == 0)

In [41]:
# print(roots)

<generator object <genexpr> at 0x7f4aec748cf0>


In [44]:
# roots = (v for v, d in G.in_degree() if d == 0)
# leaves = (v for v, d in G.out_degree() if d == 0)
# all_paths = []
# for root in roots:
#     print(root)
#     for leaf in leaves:
#         paths = nx.all_simple_paths(G, root, leaf)
#         all_paths.extend(paths)
#         break
# all_paths